In [ ]:
import pyscipopt

In [ ]:
import marimo as mo
import nbformat
import util

# MILP 定式化 (SCIP)

数理最適化モデルを用いて SCSP を解く. 以下のように定義する.

**決定変数**

- $x_{i,j} \in \mathbb{N} \ (\forall i \in \{ 1, \dots, n \}, \ \forall j \in \{ 1, \dots, |s_i| \})$: $i$ 番目の文字列の $j$ 番目の文字が解において何番目に対応するか.

**制約条件**

- $x_{i,j} < x_{i,j+1} \ (\forall i \in \{ 1, \dots, n \}, \ \forall j \in \{ 1, \dots, |s_i| - 1 \})$
- $i_1, i_2 \in \{ 1, \dots, n\} \ (i_1 \ne i_2)$ と $j_1 \in \{ 1, \dots, |s_{i_1}| \}, \ j_2 \in \{ 1, \dots, |s_{i_2}| \}$ に対し,
  $s_{i_1}[j_1] \ne s_{i_2}[j_2]$ ならば $x_{i_1, j_1} \ne x_{i_2, j_2}$.

**目的関数**

- minimize $\max_{i = 1, \dots, n} x_{i, |s_i|}$

In [ ]:
class Model:
    def __init__(self, instance: list[str]):
        max_len = sum(len(s) for s in instance)

        scip: pyscipopt.Model = pyscipopt.Model()

        seqs = [
            [
                scip.addVar(vtype="I", lb=0, ub=max_len - 1)
                for _ in s
            ]
            for s in instance
        ]
        for seq in seqs:
            for idx, _ in enumerate(seq):
                if idx == 0:
                    continue
                scip.addCons(seq[idx - 1] + 1 <= seq[idx])

        for idx1, (s1, seq1) in enumerate(zip(instance, seqs)):
            for idx2, (s2, seq2) in enumerate(zip(instance, seqs)):
                if idx1 >= idx2:
                    continue

                for cidx1, (c1, cvar1) in enumerate(zip(s1, seq1)):
                    for cidx2, (c2, cvar2) in enumerate(zip(s2, seq2)):
                        if c1 != c2:
                            lt = scip.addVar(vtype="B")
                            gt = scip.addVar(vtype="B")
                            scip.addCons(lt + gt == 1)
                            scip.addConsIndicator(cvar1 + 1 <= cvar2, binvar=lt)
                            scip.addConsIndicator(cvar1 >= cvar2 + 1, binvar=gt)

        obj = scip.addVar(vtype="C", lb=0, ub=max_len)
        for seq in seqs:
            scip.addCons(obj >= seq[-1])
        scip.setObjective(obj + 1, sense="minimize")

        self.instance = instance
        self.scip = scip
        self.seqs = seqs

    def solve(self, time_limit: int | None = 60) -> "Model":
        if time_limit is not None:
            self.scip.setParam("limits/time", time_limit)
        self.scip.optimize()

        return self

    def to_solution(self) -> str | None:
        if self.scip.getNLimSolsFound() == 0:
            return None

        objval = int(round(self.scip.getObjVal()))
        sol_char_idx = 0
        solution = ""
        while sol_char_idx <= objval:
            found = False
            for idx, (s, seq) in enumerate(zip(self.instance, self.seqs)):
                for c_idx, cvar in enumerate(seq):
                    if int(round(self.scip.getVal(cvar))) == sol_char_idx:
                        solution += s[c_idx]
                        found = True
                        sol_char_idx += 1
                if found:
                    break
            if not found:
                sol_char_idx += 1

        return solution

In [ ]:
def solve(instance: list[str], time_limit: int | None = 60) -> str | None:
    return Model(instance).solve(time_limit).to_solution()

In [ ]:
instance_01 = util.parse("uniform_q26n004k015-025.txt")
solution_01 = solve(instance_01)

presolving:
(round 1, fast)       2527 del vars, 2527 del conss, 0 add conss, 82 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 5054 impls, 0 clqs
(round 2, fast)       2527 del vars, 2527 del conss, 0 add conss, 5136 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 5054 impls, 0 clqs
(round 3, exhaustive) 2527 del vars, 2527 del conss, 0 add conss, 5136 chg bounds, 0 chg sides, 0 chg coeffs, 84 upgd conss, 5054 impls, 0 clqs
(round 4, fast)       2527 del vars, 2527 del conss, 0 add conss, 5216 chg bounds, 0 chg sides, 0 chg coeffs, 84 upgd conss, 5054 impls, 0 clqs
   (0.6s) probing: 1000/2527 (39.6%) - 0 fixings, 0 aggregations, 15588 implications, 0 bound changes
   (0.6s) probing: 1001/2527 (39.6%) - 0 fixings, 0 aggregations, 15607 implications, 0 bound changes
   (0.6s) probing aborted: 1000/1000 successive useless probings
   (0.6s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.6s) no symmetry present (symcode time

In [ ]:
_instance = instance_01
_solution = solution_01

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 70) ---
 Sol: itkgojniqfeokulvahmcnzgpibxxrndycvhodstoqgubcvsoxzvqpzprvxininsgslpbxf
str1: -tkg--n-----ku---hm----p--x--n----h---t-qg------xzv------xi---s-------
str2: i---oj-iqf-o--l-----n----bxx----cv---s----u--------qp---v-i---s-s--bxf
str3: -------------ul----c----i----n-yc--o-s-o-----v-o-z--p-p----------lp---
str4: i--g------e----va----zg--b--r-d-----d------bc-s---v----rv--n-n-g-----f

solution is feasible: True


In [ ]:
instance_02 = util.parse("uniform_q26n008k015-025.txt")
solution_02 = solve(instance_02)

presolving:


(round 1, fast)       12803 del vars, 12803 del conss, 0 add conss, 169 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 25606 impls, 0 clqs
(round 2, fast)       12803 del vars, 12803 del conss, 0 add conss, 25775 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 25606 impls, 0 clqs
(round 3, exhaustive) 12803 del vars, 12803 del conss, 0 add conss, 25775 chg bounds, 0 chg sides, 0 chg coeffs, 175 upgd conss, 25606 impls, 0 clqs
   (4.4s) probing: 1000/12803 (7.8%) - 0 fixings, 0 aggregations, 15966 implications, 0 bound changes
   (4.4s) probing: 1001/12803 (7.8%) - 0 fixings, 0 aggregations, 16000 implications, 0 bound changes
   (4.4s) probing aborted: 1000/1000 successive useless probings
   (4.4s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (4.5s) no symmetry present (symcode time: 0.02)
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 12803 deleted vars, 12803 deleted constraints, 0 added constraints, 25942 tight

In [ ]:
_instance = instance_02
_solution = solution_02

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 175) ---
 Sol: rxwxqkrdrlctodtmprpxwdenbczfjtvxerzbrvigplepbdevdcvdpfzsmsbroqvbbhpyplrzxucpmqvgtdfuivcdsboigevazgbrddbcsvrvnngfulcinycosovozpplpiojiqfolnbxxcvsuqpvissbxftkgnkuhmpxnhtqgxzvxis
str1: -----------t-----------------------------------------------------------------------------------------------------------------------------------------------kgnkuhmpxnhtqgxzvxis
str2: --------------------------------------i---------------------o----------------------------------------------------------------------jiqfolnbxxcvsuqpvissbxf---------------------
str3: -------------------------------------------------------------------------u---------------------------------------lcinycosovozpplp--------------------

In [ ]:
instance_03 = util.parse("uniform_q26n016k015-025.txt")
solution_03 = solve(instance_03)

presolving:
(round 1, fast)       46858 del vars, 46858 del conss, 0 add conss, 309 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 93716 impls, 0 clqs
(round 2, fast)       46858 del vars, 46858 del conss, 0 add conss, 94025 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 93716 impls, 0 clqs
(round 3, exhaustive) 46858 del vars, 46858 del conss, 0 add conss, 94025 chg bounds, 0 chg sides, 0 chg coeffs, 323 upgd conss, 93716 impls, 0 clqs
   (27.7s) probing: 1000/46858 (2.1%) - 0 fixings, 0 aggregations, 14925 implications, 0 bound changes
   (27.8s) probing: 1001/46858 (2.1%) - 0 fixings, 0 aggregations, 14942 implications, 0 bound changes
   (27.8s) probing aborted: 1000/1000 successive useless probings
   (27.8s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (28.2s) no symmetry present (symcode time: 0.15)
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 46858 deleted vars, 46858 deleted constraints, 0 added constra

In [ ]:
_instance = instance_03
_solution = solution_03

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution not found ---


In [ ]:
instance_04 = util.parse("uniform_q05n010k010-010.txt")
solution_04 = solve(instance_04)

presolving:
(round 1, fast)       3615 del vars, 3615 del conss, 0 add conss, 91 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 7230 impls, 0 clqs
(round 2, fast)       3615 del vars, 3615 del conss, 0 add conss, 7321 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 7230 impls, 0 clqs
(round 3, exhaustive) 3615 del vars, 3615 del conss, 0 add conss, 7321 chg bounds, 0 chg sides, 0 chg coeffs, 100 upgd conss, 7230 impls, 0 clqs
(round 4, fast)       3615 del vars, 3615 del conss, 0 add conss, 7411 chg bounds, 0 chg sides, 0 chg coeffs, 100 upgd conss, 7230 impls, 0 clqs
   (0.7s) probing: 1000/3615 (27.7%) - 0 fixings, 0 aggregations, 8604 implications, 1 bound changes
   (0.7s) probing: 1001/3615 (27.7%) - 0 fixings, 0 aggregations, 8609 implications, 1 bound changes
   (0.7s) probing aborted: 1000/1000 successive useless probings
   (0.7s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.7s) no symmetry present (symcode time

In [ ]:
_instance = instance_04
_solution = solution_04

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 45) ---
  Sol: eacbeecabcedcedabcdebceacdedbdecabecdabdceade
str01: -----------dc---bc---c---d--b--c---c-----e---
str02: ---b-------d--d-b--e--e---e---e--b--d--------
str03: --c----a-c-d-e-----e-ce-----b-e--------------
str04: -a--e------d--d---d------de-bd------d--------
str05: -acbeecabce----------------------------------
str06: ---b----b------ab--eb----d-----c-b---a-------
str07: ---b----b------a---e---a--e-b---a---da-------
str08: e---ee----e-c---b-d-b-e---e------------------
str09: --c---c----d-e-----e-----d------a---d---c--d-
str10: ---b-------d---ab-d-b-ea--------a---d--------

solution is feasible: True


In [ ]:
instance_05 = util.parse("uniform_q05n050k010-010.txt")
solution_05 = solve(instance_05)

presolving:
(round 1, fast)       98031 del vars, 98031 del conss, 0 add conss, 451 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 196062 impls, 0 clqs
(round 2, fast)       98031 del vars, 98031 del conss, 0 add conss, 196513 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 196062 impls, 0 clqs
(round 3, exhaustive) 98031 del vars, 98031 del conss, 0 add conss, 196513 chg bounds, 0 chg sides, 0 chg coeffs, 500 upgd conss, 196062 impls, 0 clqs
   (59.3s) probing: 1000/98031 (1.0%) - 0 fixings, 0 aggregations, 8616 implications, 1 bound changes
   (59.3s) probing: 1001/98031 (1.0%) - 0 fixings, 0 aggregations, 8625 implications, 1 bound changes
   (59.3s) probing aborted: 1000/1000 successive useless probings
   (59.3s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (60.3s) no symmetry present (symcode time: 0.40)
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 98031 deleted vars, 98031 deleted constraints, 0 added cons

In [ ]:
_instance = instance_05
_solution = solution_05

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution not found ---


In [ ]:
instance_06 = util.parse("nucleotide_n010k010.txt")
solution_06 = solve(instance_06)

presolving:
(round 1, fast)       3339 del vars, 3339 del conss, 0 add conss, 91 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 6678 impls, 0 clqs
(round 2, fast)       3339 del vars, 3339 del conss, 0 add conss, 6769 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 6678 impls, 0 clqs
(round 3, exhaustive) 3339 del vars, 3339 del conss, 0 add conss, 6769 chg bounds, 0 chg sides, 0 chg coeffs, 100 upgd conss, 6678 impls, 0 clqs
(round 4, fast)       3339 del vars, 3339 del conss, 0 add conss, 6859 chg bounds, 0 chg sides, 0 chg coeffs, 100 upgd conss, 6678 impls, 0 clqs
   (0.7s) probing: 1000/3339 (29.9%) - 0 fixings, 0 aggregations, 8582 implications, 1 bound changes
   (0.7s) probing: 1001/3339 (30.0%) - 0 fixings, 0 aggregations, 8589 implications, 1 bound changes
   (0.7s) probing aborted: 1000/1000 successive useless probings
   (0.7s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.7s) no symmetry present (symcode time

In [ ]:
_instance = instance_06
_solution = solution_06

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 42) ---
  Sol: TACTATATCGATCAGCTACGCTATAGTCATGCGTCATCGCTA
str01: -A-T-----G----G----G--ATA--C--G-----------
str02: -A-TA---C---C---T----T-----C---C--C-------
str03: --C-A---CGA--A--T----T---G--A-------------
str04: TA--A-A---ATC---T--G-T--------------------
str05: -A-------G----G-TA----A----CA------A-----A
str06: T--T----C---C---TA-G-----GT-A-------------
str07: T--T-----G-T-AG--A---T-----C-T------------
str08: T--------G----G----G--A-AGT--T-C----------
str09: T--T----C---CA-C-A----A----C-T------------
str10: T-CTA-A---A-C-G--A----A-------------------

solution is feasible: True


In [ ]:
instance_07 = util.parse("nucleotide_n050k050.txt")
solution_07 = solve(instance_07)

presolving:
(round 1, fast)       2283590 del vars, 2283590 del conss, 0 add conss, 2451 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 4567180 impls, 0 clqs
(round 2, fast)       2283590 del vars, 2283590 del conss, 0 add conss, 4569631 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 4567180 impls, 0 clqs
   (60.0s) probing: 4/2283590 (0.0%) - 0 fixings, 0 aggregations, 69 implications, 0 bound changes
   (60.0s) probing aborted: solving stopped
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 2283590 deleted vars, 2283590 deleted constraints, 0 added constraints, 4569631 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 4567249 implications, 0 cliques
presolving was interrupted.
Presolving Time: 37.01

SCIP Status        : solving was interrupted [time limit reached]
Solving Time (sec) : 60.03
Solving Nodes      : 0
Primal Bound       : +1.00000000000000e+20 (0 solutions)


In [ ]:
_instance = instance_07
_solution = solution_07

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

In [ ]:
instance_08 = util.parse("protein_n010k010.txt")
solution_08 = solve(instance_08)

presolving:
(round 1, fast)       4251 del vars, 4251 del conss, 0 add conss, 91 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 8502 impls, 0 clqs
(round 2, fast)       4251 del vars, 4251 del conss, 0 add conss, 8593 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 8502 impls, 0 clqs
(round 3, exhaustive) 4251 del vars, 4251 del conss, 0 add conss, 8593 chg bounds, 0 chg sides, 0 chg coeffs, 100 upgd conss, 8502 impls, 0 clqs
   (1.2s) probing: 1000/4251 (23.5%) - 0 fixings, 0 aggregations, 8421 implications, 1 bound changes
   (1.2s) probing: 1001/4251 (23.5%) - 0 fixings, 0 aggregations, 8423 implications, 1 bound changes
   (1.2s) probing aborted: 1000/1000 successive useless probings
   (1.2s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (1.3s) no symmetry present (symcode time: 0.01)
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 4251 deleted vars, 4251 deleted constraints, 0 added constraints, 8684 tightened 

In [ ]:
_instance = instance_08
_solution = solution_08

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 70) ---
  Sol: MSEFVAGVLSYCQLVPGKSSNFDAIRLNASYEHVNEQNSAHAKFTRRGTIPVQHFRKLHDLNGGYTAQNE
str01: M----A--LSYC---P-K-----------------------------GT---------------------
str02: M-----------Q-----SS------LNA--------------------IPV------------------
str03: M--------------P----------L--SY-----Q---H--F-R----------K-------------
str04: M-E----------------------------EHVNE---------------------LHD----------
str05: MS------------------NFDAIR--A----------------------------L------------
str06: M--F---------------------R-N--------QNS------R---------------NG-------
str07: M--F------Y------------A--------H------A---F---G--------------G-Y-----
str08: MS---------------K---F----------------------TRR---P-------------Y--Q--
str09: MS-FVAGV------------------------------------

In [ ]:
instance_09 = util.parse("protein_n050k050.txt")
solution_09 = solve(instance_09)

presolving:
presolving (0 rounds: 0 fast, 0 medium, 0 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolving was interrupted.
Presolving Time: 0.60

SCIP Status        : solving was interrupted [time limit reached]
Solving Time (sec) : 76.77
Solving Nodes      : 0
Primal Bound       : +1.00000000000000e+20 (0 solutions)


In [ ]:
_instance = instance_09
_solution = solution_09

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS